# Part 1

In [10]:
import re

line_re = re.compile(r'^pos=<(-?\d+),(-?\d+),(-?\d+)>, r=(\d+)$')

def parse_text(text):
    ''' Parse text and return list of tuples (x,y,z,r). '''
    nanobots = list()
    for line in text.strip().split('\n'):
        match = line_re.match(line)
        if not match:
            raise Exception('Could not match line: {}'.format(line))
        x, y, z, r = match.group(1,2,3,4)
        nanobots.append((int(x), int(y), int(z), int(r)))
    return nanobots

In [11]:
test_text = '''pos=<0,0,0>, r=4
pos=<1,0,0>, r=1
pos=<4,0,0>, r=3
pos=<0,2,0>, r=1
pos=<0,5,0>, r=3
pos=<0,0,3>, r=1
pos=<1,1,1>, r=1
pos=<1,1,2>, r=1
pos=<1,3,1>, r=1
'''

In [12]:
nanobots = parse_text(test_text)
print(nanobots)

[(0, 0, 0, 4), (1, 0, 0, 1), (4, 0, 0, 3), (0, 2, 0, 1), (0, 5, 0, 3), (0, 0, 3, 1), (1, 1, 1, 1), (1, 1, 2, 1), (1, 3, 1, 1)]


In [13]:
# We can get the largest radius by sorting and taking the first item.
nanobots.sort(key=lambda n: n[3], reverse=True)
maxbot = nanobots[0]
print(maxbot)

(0, 0, 0, 4)


In [14]:
def filter_bots(bot1, nanobots):
    count = 0
    x1, y1, z1, r = bot1
    for x2, y2, z2, _ in nanobots:
        if abs(x1-x2) + abs(y1-y2) + abs(z1-z2) <= r:
            count += 1
    return count

In [15]:
filter_bots(maxbot, nanobots)

7

In [16]:
with open('input.txt') as input_:
    text = input_.read()
nanobots = parse_text(text)
print(len(nanobots))
print(nanobots[-1])

1000
(75613424, 50145291, 41345801, 72623408)


In [17]:
# Get nanobot with largest radius.
nanobots.sort(key=lambda n: n[3], reverse=True)
maxbot = nanobots[0]
print(maxbot)

(-19184742, 69238414, 51896760, 99428620)


In [18]:
filter_bots(maxbot, nanobots)

599

# Part 2

The naive solution is to construct a 3D sparse matrix that counts how many nanobots are in range of each cell, but that won't work here because the average radius of a nanobot is about 50,000,000. The number of cells to fill in for each bot is (I think) roughly `radius ^ 3 / 3`, which is an absolutely huge number–way bigger than I could possibly fit in memory.

So I think a better alternative is to try hill climbing: starting from an arbitrary point, move 1 unit in any direction that improves my objective function. I just need to define what objective function to use.

I already have a `filter_bots()` function that could be used as an objective function, but `filter_bots()` is very discontinuous: you might have to move millions of units to produce a 1 unit change in `filter_bots()`. So instead, my score is going to be based on how far a point is to the edge of the area covered by a nanobot. This way, the score improves even when you move just 1 unit, and I imagine that optimizing this score would be equivalent to optimizing the score in `filter_bots()`.

In [48]:
test_text = '''pos=<10,12,12>, r=2
pos=<12,14,12>, r=2
pos=<16,12,12>, r=4
pos=<14,14,14>, r=6
pos=<50,50,50>, r=200
pos=<10,10,10>, r=5
'''
nanobots = parse_text(test_text)
print(nanobots)

[(10, 12, 12, 2), (12, 14, 12, 2), (16, 12, 12, 4), (14, 14, 14, 6), (50, 50, 50, 200), (10, 10, 10, 5)]


In [49]:
def dist_to_nanobot(x, y, z, nanobot):
    ''' Compute the manhattan distance from x,y,z to the closest point within
    range of the given nanobot. If x,y,z is already in the nanobot's range, return
    zero. '''
    # The distance is equal to the distance from x,y,z to the 
    # nanobot's center minus the nanobot's radius.
    nx, ny, nz, nr = nanobot
    dx = abs(nx - x)
    dy = abs(ny - y)
    dz = abs(nz - z)
    return max(dx + dy + dz - nr, 0)

In [50]:
print(nanobots[0])

(10, 12, 12, 2)


In [51]:
# These coordinates are within the nanobots range, so the distance is zero.
assert dist_to_nanobot(10, 12, 12, nanobots[0]) == 0
assert dist_to_nanobot(11, 13, 12, nanobots[0]) == 0

In [52]:
# A few more tests...
assert dist_to_nanobot(18, 12, 12, nanobots[0]) == 6
assert dist_to_nanobot(9, 11, 11, nanobots[0]) == 1
assert dist_to_nanobot(11, 13, 13, nanobots[0]) == 1
assert dist_to_nanobot(0, 0, 0, nanobots[0]) == 32

In [53]:
def filter_bots2(x1, y1, z1, nanobots):
    count = 0
    for x2, y2, z2, r in nanobots:
        if abs(x1-x2) + abs(y1-y2) + abs(z1-z2) <= r:
            count += 1
    return count

In [54]:
# The problem says there are 5 nanobots within range of this coordinate.
filter_bots2(12, 12, 12, nanobots)

5

In [55]:
def objective(x, y, z, nanobots):
    ''' This is the function we want to minimize: the sum of the distances
    from x,y,z to all nanobots. '''
    sum_ = 0
    for nanobot in nanobots:
        sum_ += dist_to_nanobot(x, y, z, nanobot)
    return sum_

In [56]:
# The origin is clearly not a good solution.
objective(0, 0, 0, nanobots)

165

In [57]:
# We know from the problem statement that 12,12,12 is optimal.
objective(12, 12, 12, nanobots)

1

In [58]:
import math

def hill_climb(x, y, z, nanobots, step=1, debug=False):
    ''' Given an initial x,y,z, perform hill climbing algorithm to find
    optimal coordinates. '''
    old_cost = math.inf
    new_cost = objective(x, y, z, nanobots)
    while True:
        neighbors = (
            (x-step, y     ,  z   ),(x+step, y     , z     ),
            (x     , y-step,  z   ),(x     , y+step, z     ),
            (x     , y     ,z-step),(x     , y     , z+step),
        )
        options = [(objective(nx,ny,nz,nanobots),nx,ny,nz) for nx,ny,nz in neighbors]
        best = min(options)
        old_cost = new_cost
        new_cost = best[0]
        if new_cost >= old_cost:
            return x, y, z, old_cost 
        x, y, z = best[1:4]
        if debug:
            in_range = filter_bots2(x, y, z, nanobots)
            print('cost={} in_range={} coord={},{},{}'.format(
                new_cost, in_range, x, y, z))

In [59]:
hill_climb(0, 0, 0, nanobots, debug=True)

cost=160 in_range=1 coord=0,0,1
cost=155 in_range=1 coord=0,0,2
cost=150 in_range=1 coord=0,0,3
cost=145 in_range=1 coord=0,0,4
cost=140 in_range=1 coord=0,0,5
cost=135 in_range=1 coord=0,0,6
cost=130 in_range=1 coord=0,0,7
cost=125 in_range=1 coord=0,0,8
cost=120 in_range=1 coord=0,0,9
cost=115 in_range=1 coord=0,0,10
cost=110 in_range=1 coord=0,1,10
cost=105 in_range=1 coord=0,2,10
cost=100 in_range=1 coord=0,3,10
cost=95 in_range=1 coord=0,4,10
cost=90 in_range=1 coord=0,5,10
cost=85 in_range=1 coord=0,6,10
cost=80 in_range=1 coord=0,7,10
cost=75 in_range=1 coord=0,8,10
cost=70 in_range=1 coord=0,9,10
cost=65 in_range=1 coord=0,10,10
cost=60 in_range=1 coord=1,10,10
cost=55 in_range=1 coord=2,10,10
cost=50 in_range=1 coord=3,10,10
cost=45 in_range=1 coord=4,10,10
cost=40 in_range=2 coord=5,10,10
cost=36 in_range=2 coord=6,10,10
cost=32 in_range=2 coord=6,10,11
cost=28 in_range=2 coord=7,10,11
cost=24 in_range=2 coord=7,10,12
cost=20 in_range=2 coord=8,10,12
cost=16 in_range=2 coord=

(12, 12, 12, 1)

In [60]:
%%time
# Hill climbing finds the optimal solution given in the practice
# problem when starting from the origin, but I wonder how well it works
# when starting from other places?
print('Starting from nanobots[0]', hill_climb(*nanobots[0][1:], nanobots))
print('Starting from nanobots[1]', hill_climb(*nanobots[1][1:], nanobots))
print('Starting from nanobots[2]', hill_climb(*nanobots[2][1:], nanobots))
print('Starting from 100,100,100', hill_climb(100,100,100, nanobots))
print('Starting from 999,9999,99999', hill_climb(999,9999,99999, nanobots))

Starting from nanobots[0] (12, 12, 12, 1)
Starting from nanobots[1] (12, 12, 12, 1)
Starting from nanobots[2] (12, 12, 12, 1)
Starting from 100,100,100 (12, 12, 12, 1)
Starting from 999,9999,99999 (12, 12, 12, 1)
CPU times: user 3.33 s, sys: 4.58 ms, total: 3.33 s
Wall time: 3.33 s


In [61]:
# Great, so hill climbing seems to work pretty well on the example problem, even
# with poorly chosen seeds. Now I'll try this on the real problem. I'll run it
# a couple of times using various nanobots as seeds.
with open('input.txt') as input_:
    text = input_.read()
nanobots = parse_text(text)
print(len(nanobots))

1000


In [88]:
# With the default step size of 1, hill climbing runs for several minutes
# without finding a solution. So I'm going to try running a large step size
# to find a coarse result, then using successively smaller step sizes to
# refine the solution.
def fast_hill_climb(x, y, z, nanobots, debug=False):
    step = 100_000
    best = (x, y, z, math.inf)
    while step >= 1:
        best = hill_climb(*best[0:3], nanobots, step=step)
        in_range = filter_bots2(*best[0:3], nanobots)
        if debug:
            print('step={} best={} in_range={}'.format(step, best, in_range))
        step = step // 10
    return in_range, best

In [74]:
%time fast_hill_climb(*nanobots[314][1:], nanobots, debug=True)

step=100000 best=(26721486, 46587696, 21148148, 3283181627) in_range=878
step=10000 best=(26791486, 46587696, 21098148, 3279301627) in_range=878
step=1000 best=(26794486, 46606696, 21079148, 3278215754) in_range=901
step=100 best=(26794886, 46607396, 21078848, 3278185741) in_range=950
step=10 best=(26794876, 46607406, 21078798, 3278185276) in_range=955
step=1 best=(26794876, 46607405, 21078801, 3278185269) in_range=955
CPU times: user 3.51 s, sys: 3.69 ms, total: 3.52 s
Wall time: 3.52 s


(26794876, 46607405, 21078801, 3278185269)

In [75]:
%time fast_hill_climb(*nanobots[728][1:], nanobots, debug=True)

step=100000 best=(26727744, 46566110, 21147648, 3283428747) in_range=878
step=10000 best=(26787744, 46606110, 21087648, 3278757385) in_range=793
step=1000 best=(26793744, 46607110, 21079648, 3278245348) in_range=900
step=100 best=(26794844, 46607410, 21078848, 3278185711) in_range=945
step=10 best=(26794874, 46607400, 21078808, 3278185272) in_range=952
step=1 best=(26794874, 46607401, 21078807, 3278185270) in_range=953
CPU times: user 5.36 s, sys: 6.95 ms, total: 5.36 s
Wall time: 5.36 s


(26794874, 46607401, 21078807, 3278185270)

In [76]:
%time fast_hill_climb(*nanobots[152][1:], nanobots, debug=True)

step=100000 best=(26706930, 46546906, 21147114, 3284542891) in_range=878
step=10000 best=(26786930, 46606906, 21087114, 3278704971) in_range=885
step=1000 best=(26794930, 46606906, 21079114, 3278220321) in_range=827
step=100 best=(26794830, 46607306, 21078914, 3278187450) in_range=920
step=10 best=(26794870, 46607396, 21078814, 3278185294) in_range=948
step=1 best=(26794874, 46607401, 21078807, 3278185270) in_range=953
CPU times: user 2.75 s, sys: 4.29 ms, total: 2.76 s
Wall time: 2.75 s


(26794874, 46607401, 21078807, 3278185270)

In [77]:
%time fast_hill_climb(*nanobots[516][1:], nanobots, debug=True)

step=100000 best=(26755751, 43746767, 23918362, 3483266196) in_range=859
step=10000 best=(26785751, 46606767, 21088362, 3278785065) in_range=885
step=1000 best=(26794751, 46606767, 21079362, 3278212732) in_range=908
step=100 best=(26794851, 46607367, 21078862, 3278185792) in_range=940
step=10 best=(26794871, 46607397, 21078812, 3278185285) in_range=950
step=1 best=(26794874, 46607401, 21078807, 3278185270) in_range=953
CPU times: user 6.97 s, sys: 13.7 ms, total: 6.99 s
Wall time: 7 s


(26794874, 46607401, 21078807, 3278185270)

In [78]:
%time fast_hill_climb(*nanobots[928][1:], nanobots, debug=True)

step=100000 best=(26710180, 46546964, 21173077, 3285210005) in_range=878
step=10000 best=(26790180, 46606964, 21083077, 3278472302) in_range=896
step=1000 best=(26794180, 46606964, 21079077, 3278216399) in_range=898
step=100 best=(26794880, 46607364, 21078877, 3278186329) in_range=942
step=10 best=(26794880, 46607404, 21078807, 3278185272) in_range=954
step=1 best=(26794878, 46607404, 21078806, 3278185269) in_range=954
CPU times: user 7.27 s, sys: 19.6 ms, total: 7.28 s
Wall time: 7.3 s


(26794878, 46607404, 21078806, 3278185269)

In [79]:
%time fast_hill_climb(*nanobots[435][1:], nanobots, debug=True)

step=100000 best=(26755059, 45410067, 22240840, 3340910479) in_range=877
step=10000 best=(26785059, 46600067, 21080840, 3278775030) in_range=813
step=1000 best=(26794059, 46607067, 21079840, 3278240937) in_range=886
step=100 best=(26794859, 46607367, 21078840, 3278185591) in_range=938
step=10 best=(26794869, 46607397, 21078810, 3278185288) in_range=948
step=1 best=(26794874, 46607401, 21078807, 3278185270) in_range=953
CPU times: user 4.52 s, sys: 12 ms, total: 4.53 s
Wall time: 4.54 s


(26794874, 46607401, 21078807, 3278185270)

In [80]:
%time fast_hill_climb(0, 0, 0, nanobots, debug=True)

step=100000 best=(21300000, 41100000, 21100000, 3776591533) in_range=855
step=10000 best=(26790000, 46600000, 21080000, 3278716070) in_range=800
step=1000 best=(26795000, 46607000, 21079000, 3278224548) in_range=821
step=100 best=(26794800, 46607400, 21078900, 3278186896) in_range=933
step=10 best=(26794880, 46607410, 21078800, 3278185271) in_range=955
step=1 best=(26794880, 46607409, 21078801, 3278185269) in_range=955
CPU times: user 9.5 s, sys: 21 ms, total: 9.52 s
Wall time: 9.53 s


(26794880, 46607409, 21078801, 3278185269)

In [81]:
%time fast_hill_climb(*nanobots[986][1:], nanobots, debug=True)

step=100000 best=(26687434, 46531917, 21103011, 3285185832) in_range=807
step=10000 best=(26787434, 46601917, 21083011, 3278672423) in_range=878
step=1000 best=(26794434, 46602917, 21083011, 3278414171) in_range=890
step=100 best=(26794834, 46607317, 21078911, 3278187175) in_range=923
step=10 best=(26794874, 46607397, 21078811, 3278185279) in_range=951
step=1 best=(26794874, 46607401, 21078807, 3278185270) in_range=953
CPU times: user 7.53 s, sys: 10.4 ms, total: 7.54 s
Wall time: 7.54 s


(26794874, 46607401, 21078807, 3278185270)

In [82]:
%time fast_hill_climb(*nanobots[26][1:], nanobots, debug=True)

step=100000 best=(26720226, 46522361, 21128104, 3284060517) in_range=878
step=10000 best=(26790226, 46602361, 21078104, 3278595453) in_range=738
step=1000 best=(26794226, 46607361, 21079104, 3278234959) in_range=819
step=100 best=(26794826, 46607361, 21078804, 3278186059) in_range=938
step=10 best=(26794876, 46607401, 21078804, 3278185270) in_range=954
step=1 best=(26794876, 46607401, 21078805, 3278185269) in_range=954
CPU times: user 4.67 s, sys: 5.99 ms, total: 4.67 s
Wall time: 4.67 s


(26794876, 46607401, 21078805, 3278185269)

In [45]:
# The two best outcomes of the runs above (both within range of 955 nanobots) are:
print(26794880 + 46607409 + 21078801)
print(26794876 + 46607405 + 21078801)
# The first one is closer, but when I tried it as an answer, it said it
# was too low! Interesting... so there is a better solution (>955) somewhere
# farther away from the origin.

94481082
94481088


In [94]:
%%time
# Now I'll try evaluating EVERY nanobot as a seed. This should take about
# an hour, but my laptop went to sleep in the middle so it ended up taking
# longer.
results = list()
for i, nanobot in enumerate(nanobots):
    results.append(fast_hill_climb(*nanobot[1:], nanobots))
    if i % 50 == 0:
        print('finished', i)
results.sort()
print(results[:10])

finished 0
finished 50
finished 100
finished 150
finished 200
finished 250
finished 300
finished 350
finished 400
finished 450
finished 500
finished 550
finished 600
finished 650
finished 700
finished 750
finished 800
finished 850
finished 900
finished 950
[(953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270)), (953, (26794874, 46607401, 21078807, 3278185270))]
CPU times: user 3h 11min 32s, sys: 7.04 s, total: 3h 11min 39s
Wall time: 6h 40min 25s


In [95]:
# Whoops, forgot to sort descending.
results.sort(reverse=True)
print(results[:10])

[(957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607411, 21078799, 3278185272)), (957, (26794881, 46607410, 21078800, 3278185270))]


In [100]:
# The previous best was 955 and this set of results has at least 
# 10 results for 957. Let's pick the one closest to the origin.
scored_results = [(abs(x)+abs(y)+abs(z), in_range, x, y, z) 
                  for in_range,(x,y,z,dist) in results if in_range>=957]
scored_results.sort()
print(scored_results)

[(94481091, 957, 26794881, 46607410, 21078800), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799), (94481091, 957, 26794881, 46607411, 21078799)]


At this point, I'm completely stumped on this problem, so [I turned to Reddit](https://www.reddit.com/r/adventocode) to find tips. There are a bunch of esoteric solutions that depend on external SAT solvers or graph theory algorithms, but I want to keep my solutions purely inside the Python standard library.

The approach I like best uses space partionining: break the entire search space into 8 equal size cubes and count how many nanobots intersect with each cube. Pick the cube with the highest number of intersections and break it into 8 smaller cubes, repeating the process until you find a cube that is only 1 unit^3 in volume, and that is the answer. So I will try to implement that approach here.

In [120]:
# First, I want to find a power-of-2 bounding box that contains all
# of the nanobots, and to make things simple, I want it to be centered
# on the origin.
min_coord, max_coord = math.inf, -math.inf
for nanobot in nanobots:
    min_coord = min(min_coord, nanobot[0] - nanobot[3], 
        nanobot[1] - nanobot[3], nanobot[0] - nanobot[3])
    max_coord = max(max_coord, nanobot[0] + nanobot[3], 
        nanobot[1] + nanobot[3], nanobot[0] + nanobot[3])
print('coords', min_coord, max_coord)

coords -256167833 291969427


In [122]:
# A cube that is 2^29 on each side would cover 1/8th of 
# the search space, therefore a 2^30 cube would cover all of it.
math.log(291969427, 2)

28.121242067160633

In [224]:
from collections import namedtuple

# A search cube contains the coordinate of its corner (the one with 
# the smallest 3 coordinates, and a  a power-of-2 length of its edges.
# It stores the number of bots that intersect it and the L1 distance
# from the center of the cube to the origin. The number of bots is
# inverted and the fields are ordered so that we can use Python's minheap
# as a priority queue.
SearchCube = namedtuple('SearchCube', 'bot_count dist x y z length')

def make_cube(x, y, z, length, nanobots):
    ''' Given a corner of a cube and a length, return a SearchCube object. '''
    bot_count = 0
    min_x, max_x = x, x + length
    min_y, max_y = y, y + length
    min_z, max_z = z, z + length
    cube_corners = (
        (x         , y         , z         ),
        (x+length-1, y         , z         ),
        (x         , y+length-1, z         ),
        (x         , y         , z+length-1),
        (x+length-1, y+length-1, z         ),
        (x         , y+length-1, z+length-1),
        (x+length-1, y         , z+length-1),
        (x+length-1, y+length-1, z+length-1),
    )
    # Count nanobots that intersect this cube.
    for nx, ny, nz, nr in nanobots:
        # Check if a nanobot corner is in this cube
        corners = ((nx-r, ny  , nz  ), (nx+r, ny   , nz  ),
                   (nx  , ny-r, nz  ), (nx  , ny+r , nz  ),
                   (nx  , ny  , nz-r), (nx  , ny   , nz+r))
        for px, py, pz in corners:
            if ((min_x <= px <= max_x) and
                (min_y <= py <= max_y) and
                (min_z <= pz <= max_z)):
                bot_count -= 1
                break
        else:
            # Check if a cube corner is in range of a nanobot
            for cx, cy, cz in cube_corners:
                if abs(nx-cx) + abs(ny-cy) + abs(nz-cz) <= nr:
                    bot_count -= 1
                    break
    dist = (x + length // 2) + (y + length // 2) + (z + length // 2)
    return SearchCube(bot_count, dist, x, y, z, length)

In [225]:
make_cube(10,10,10, 1, [])

SearchCube(bot_count=0, dist=30, x=10, y=10, z=10, length=1)

In [226]:
make_cube(0,0,0,32,[])

SearchCube(bot_count=0, dist=48, x=0, y=0, z=0, length=32)

In [227]:
# This is a cube that contains the centers of all nanobots.
min_x = min(n[0] for n in nanobots)
max_x = max(n[0] for n in nanobots)
min_y = min(n[1] for n in nanobots)
max_y = max(n[1] for n in nanobots)
min_z = min(n[2] for n in nanobots)
max_z = max(n[2] for n in nanobots)
length = max(max_x-min_x, max_y-min_y, max_z-min_z)
make_cube(min_x, min_y, min_z, length, nanobots)

SearchCube(bot_count=-1000, dist=178846919, x=-178506040, y=-141655043, z=-96576988, length=397056660)

In [228]:
# According to my previous results, this point is in range of 957 nanobots.
make_cube(26794881, 46607411, 21078799, 1, nanobots)

SearchCube(bot_count=-957, dist=94481091, x=26794881, y=46607411, z=21078799, length=1)

In [229]:
# This will be the cube I use to start the search. It contains all
# 1000 nanobots and is centered on the origin.
make_cube(-2**29, -2**29, -2**29, 2**30, nanobots)

SearchCube(bot_count=-1000, dist=0, x=-536870912, y=-536870912, z=-536870912, length=1073741824)

In [231]:
from heapq import heappush, heappop, nlargest
import itertools

def debug_heap(h):
    h = list(h)
    print(heappop(h), heappop(h), heappop(h))
    
def partition_solve(nanobots):
    ''' Find optimal point using space partioning. '''
    megacube = make_cube(-2**29, -2**29, -2**29, 2**30, nanobots)
    heap = [megacube]
    def subcube(x, y, z, l):
        ''' Helper for making a cube and putting it on the heap. '''
        heappush(heap, make_cube(x, y, z, l, nanobots))
    for n in itertools.count():
        if n % 1000 == 0:
            print('n={} #heap={}'.format(n, len(heap)))
        cube = heappop(heap)
        if cube.length == 1:
            return cube
        # Break up into 8 smaller cubes and push them onto heap.
        length = cube.length // 2
        x, y, z = cube.x, cube.y, cube.z
        subcube(x       , y       , z       , length)
        subcube(x+length, y       , z       , length)
        subcube(x       , y+length, z       , length)
        subcube(x       , y       , z+length, length)
        subcube(x+length, y+length, z       , length)
        subcube(x       , y+length, z+length, length)
        subcube(x+length, y       , z+length, length)
        subcube(x+length, y+length, z+length, length)
#         print('n={} #heap={}'.format(n, len(heap)))
#         debug_heap(heap)
#         if n > 3:
#             break

In [232]:
%time solution = partition_solve(nanobots)
print(solution)

n=0 #heap=1
CPU times: user 3.28 s, sys: 8.62 ms, total: 3.29 s
Wall time: 3.29 s
SearchCube(bot_count=-977, dist=94481130, x=26794906, y=46607439, z=21078785, length=1)


In [233]:
# Check to make sure that filter_bots2() agrees with the bot count.
filter_bots2(solution.x, solution.y, solution.z, nanobots)

977

In [234]:
print(solution.x + solution.y + solution.z)

94481130
